In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Ryan.Models.LSTM_model_Regression.preprocess import preprocess_data
from Ryan.Models.LSTM_model_Regression.lstm_models import  LSTMModel, LSTMModel_multi
from Ryan.Models.LSTM_model_Regression.run_fn import eval_fn, train_fn
from Ryan.Models.LSTM_model_Regression.load_data import load_data, create_sequences
from Ryan.Models.LSTM_model_Regression.load_data import load_data2
import torch
from transformers import get_linear_schedule_with_warmup
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

# 1. Load Data
use the `resampled_lob_secs2.csv` file as the test data (last 10%)

align the test index with the prediction index. The length of both should be the same (376707)

set `test_set=test_data[21:-1]` to align the test data with the prediction data

In [ ]:
lob_data=pd.read_csv('E:\\Bristol\\mini_project\\JPMorgan_Set01\\test_datasets\\resampled_lob_secs2.csv')

loaded_pred=np.load('./saved_models_regression/predictions_new3.npy')
loaded_true=np.load('./saved_models_regression/actuals_new3.npy')

In [ ]:
split_index=int(0.9*len(lob_data))
train_data=lob_data.iloc[:split_index]
test_data=lob_data.iloc[split_index:]
test_set=test_data[21:-1]

test_set.reset_index(drop=True, inplace=True) # drop=True to avoid the old index being added as a column
test_set

In [ ]:
# use the true price in test set and predicted price to plot
plt.figure(figsize=(10,6))
plt.plot(test_set['price'][300000:300100].values, label='Actuals')
plt.plot(loaded_pred[300000:300100], label='Predictions')
plt.legend()
plt.title('Predictions vs Actuals of LSTM model')
plt.ylabel('Price')
plt.xlabel('Samples')
plt.show()

# 2. Trading Strategy

In [ ]:
df=pd.DataFrame()
df['ori_price']=test_set['price']
df['pred']=loaded_pred
df

In [ ]:
# This is only for trading volume, not for modeling
# # 设置一个时间窗口
rolling_window = 5  # 例如，我们使用过去5个时间点的数据来计算波动率

# 计算对数收益率 shift(1)是为了计算相对于前一分钟的收益率
test_set['Log Return Max Bid'] = np.log(test_set['Max Bid Price'] / test_set['Max Bid Price'].shift(1))
test_set['Log Return Min Ask'] = np.log(test_set['Min Ask Price'] / test_set['Min Ask Price'].shift(1))

# 计算滚动标准差作为波动率的度量
test_set['Volatility Max Bid'] = test_set['Log Return Max Bid'].rolling(window=rolling_window).std()
test_set['Volatility Min Ask'] = test_set['Log Return Min Ask'].rolling(window=rolling_window).std()

# 由于滚动计算会产生缺失值，我们通常会删除这些值
# lob_data.dropna(inplace=True)

# Trading volume
def adjust_trade_quantity(volatility, max_tradeable_quantity, base_quantity=1, risk_tolerance=0.5, scaler_factor=10):
    """
    根据波动率和最大可交易量调整交易量。
    volatility: 当前波动率
    max_tradeable_quantity: 该时间点的最大可交易量（对于买入操作，是Min Ask Quantity；对于卖出操作，是Max Bid Quantity）
    base_quantity: 基础交易量
    risk_tolerance: 风险容忍度，取值范围为[0, 1]，数值越小表风险承受越大，交易量越大
    scaler_factor: 缩放因子，用于进一步调整基于波动率的交易量
    """
    # 确保风险容忍度和波动率不会导致除以0
    if volatility <= 0.0001:
        volatility = 0.0001
    if risk_tolerance <= 0:
        risk_tolerance = 0.1  # 设置一个默认的风险容忍度，避免除以0

    # 基于波动率调整的交易量
    adjusted_quantity = base_quantity / (volatility * risk_tolerance * scaler_factor)
    adjusted_quantity = max(1, round(adjusted_quantity))  # 确保至少交易1单位，并且是整数

    # 确保交易量不超过最大可交易量
    final_trade_quantity = min(adjusted_quantity, max_tradeable_quantity)

    return final_trade_quantity

In [37]:
initial_capital = 100000
capital = initial_capital
has_position = False
trade_log = []
# capital_history = [initial_capital]
current_holdings = 0
transaction_cost = 0.01
# trade_quantity = 1

count=0


for i in range(len(test_set)-1):
    # skip the iteration if there is any missing value
    count+=1
    if test_set.iloc[i].isna().any():
        continue

    current_predict = loaded_pred[i]
    next_predict = loaded_pred[i+1]
    current_true = test_set['price'][i]
    # predict_price = y_pred_rf[i + 1]
    # current_price = y_pred_rf[i]
    
    # adjust the trade quantity based on the volatility. make the trade quantity no more than the max tradeable quantity
    current_volatility_buy = test_set['Volatility Min Ask'].iloc[i ]
    current_volatility_sell = test_set['Volatility Max Bid'].iloc[i ]
    ofi = test_set['ofi'].iloc[i]
    rsi = test_set['RSI'].iloc[i]

    # buy policy: 当下一个点预测价格比当前点预测价格上升大于0.5%时，买入
    if not has_position and next_predict > current_predict * 1.005:
        # if not has_position and rsi>60:

        # buy_price=X_test['price'].iloc[i]
        # buy_price = lob_data['price'].iloc[i + split_index]
        buy_price = current_true
        buy_quantity = adjust_trade_quantity(current_volatility_buy, test_set['Min Ask Quantity'].iloc[i],
                                             scaler_factor=10)
        capital -= buy_price * buy_quantity * (1 + transaction_cost)
        current_holdings += buy_quantity
        has_position = True
        trade_log.append({'action': 'BUY1', 'Time': test_set['Datetime'].iloc[i], 'price': buy_price,
                          'capital': capital, 'quantity': buy_quantity,
                          'max quantity': test_set['Min Ask Quantity'].iloc[i]})
     
    # buy 2: when has position but next price is much higher than current price, buy more (5% increase)   
    if has_position and next_predict> current_predict*1.05: 
        buy_price = current_true
        buy_quantity = adjust_trade_quantity(current_volatility_buy, test_set['Min Ask Quantity'].iloc[i],
                                             scaler_factor=10)
        capital -= buy_price * buy_quantity * (1 + transaction_cost)
        current_holdings += buy_quantity
        has_position = True
        trade_log.append({'action': 'BUY2', 'Time': test_set['Datetime'].iloc[i], 'price': buy_price,
                          'capital': capital, 'quantity': buy_quantity,
                          'max quantity': test_set['Min Ask Quantity'].iloc[i]})

    # sell policy: 当下一个点预测价格比当前点预测价格下降大于0.5%时，卖出
    elif has_position and next_predict < current_predict * 0.995:
        # elif has_position and rsi<40:

        # sell_price=X_test['price'].iloc[i]
        # sell_price = lob_data['price'].iloc[i + split_index]
        sell_price = current_true
        sell_quantity = min(current_holdings, adjust_trade_quantity(current_volatility_sell,
                                                                    test_set['Max Bid Quantity'].iloc[i],
                                                                    scaler_factor=20))
        capital += sell_price * sell_quantity * (1 - transaction_cost)
        current_holdings -= sell_quantity
        
        if current_holdings == 0:
            has_position = False
        
        trade_log.append({'action': 'SELL', 'Time': test_set['Datetime'].iloc[i], 'price': sell_price,
                          'capital': capital, 'quantity': sell_quantity,
                          'max quantity': test_set['Max Bid Quantity'].iloc[i]})

    # hold policy: 当下一个点预测价格比当前点预测价格波动小于0.5%时，继续持有
    elif has_position and current_predict * 0.995 <= next_predict <= current_predict * 1.005:
        trade_log.append({'action': 'HOLD', 'Time': test_set['Datetime'].iloc[i],
                          'price': test_set['price'].iloc[i], 'capital': capital,
                          'quantity': current_holdings})

# 计算最终资本
final_capital = capital
profit = final_capital - initial_capital
# 只计算buy和sell的次数
number_of_buy = len([trade for trade in trade_log if trade['action'] in ['BUY1','BUY2']])
number_of_sell = len([trade for trade in trade_log if trade['action'] == 'SELL'])
# number_of_trades = len(trade_log)

# 保留两位小数
# print('Final capital: ', round(final_capital))
print('Profit: ', round(profit))
# print('Final return: ', round(profit / initial_capital * 100, 2), '%')
print('Number of buys: ', number_of_buy)
print('Number of sells: ', number_of_sell)

Profit:  7208478
Number of buys:  139302
Number of sells:  126278


### EXP1
buy: next_predict > current_predict * 1.005
sell: next_predict < current_predict * 0.995
hold: current_predict * 0.995 <= next_predict <= current_predict * 1.005

Profit:  3794389
Number of trades:  181696

### EXP2
buy1: next_predict > current_predict * 1.005 (0.5% increase)
buy2: has_position and next_predict> current_predict*1.05 (5% increase)
sell: next_predict < current_predict * 0.995
hold: current_predict * 0.995 <= next_predict <= current_predict * 1.005

Profit:  7208478
Number of buys:  139302
Number of sells:  126278